In [ ]:
#check state
import json
import pandas as pd
totally_random = 42
try:
  assert 'authors' in globals(), "no data to work with, run previous cell or place intermediate data in data/intermediate"
  data_path_val = f'{folder}/data/val_data_realvalued.csv'
  data_path_out = f'{folder}/data/intermediate/humility_added.json'
  from_nb = 'var'
except:
  try:
    data_path_in = f'{folder}/data/intermediate/langdetect.json'
    data_path_val = f'{folder}/data/val_data_realvalued.csv'
    data_path_out = f'{folder}/data/intermediate/humility_added.json'
    from_nb = 'save'
  except:
    folder_local = '/content/drive/MyDrive/digital text analysis/NLP/shared task'
    data_path_in = f'{folder_local}/data/intermediate/langdetect.json'
    data_path_out = f'{folder_local}/data/intermediate/humility_added.json'
    data_path_val = f'{folder_local}/data/val_data_realvalued.csv'
    from_nb = 'local'

try:
  assert 'skip_gated_model' in globals(), 'pass'
except:
  skip_gated_model = False

#load
if from_nb == 'var':
  pass
elif from_nb in ['save', 'local']:
  try:
    with open(data_path_in, 'r', encoding='utf-8') as f:
      authors = json.load(f)
  except:
    raise Exception('no authors variable or save detected')
else:
  raise Exception('no authors variable or save detected')

print(f'NB loaded in {from_nb}')


NB loaded in local


Scoring users on humility

In [ ]:
if skip_gated_model == False:
  %pip install outlines
  import json
  import pandas as pd
  import torch
  import outlines
  import random


  #config
  comment_limit = 5
  random.seed(totally_random)

  #few shot examples
  #get random examples every time.
  df = pd.read_csv(data_path_val)
  df_sorted = df.sort_values(by='Humility').reset_index(drop=True)
  low_humility_start_idx = 0
  medium_humility_start_idx = df_sorted[df_sorted['Humility'] > 33].index[0]
  high_humility_start_idx = df_sorted[df_sorted['Humility'] > 66].index[0]
  def get_random_examples(df_sorted):
    few_shot_examples = []
    rand_low_idx = random.choice(range(low_humility_start_idx,medium_humility_start_idx))
    rand_med_idx = random.choice(range(medium_humility_start_idx,high_humility_start_idx))
    rand_hi_idx = random.choice(range(high_humility_start_idx,len(df_sorted)))
    selected_idx = [rand_low_idx,rand_med_idx,rand_hi_idx]
    selected_rows = df_sorted.iloc[selected_idx]
    for index, row in selected_rows.iterrows():
      value_tuple = ([row['Q1'], row['Q2'], row['Q3']],row['Humility']/100)
      few_shot_examples.append(value_tuple)
    random.shuffle(few_shot_examples)
    return few_shot_examples

  #model
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f'device: {device}')
  model_name = "meta-llama/Llama-3.2-3B-Instruct"
  model = outlines.models.transformers(model_name, device=device)
  pattern = r'0\.\d{2}|1\.00'
  generator = outlines.generate.regex(model, pattern)

  #prompt
  prompt = f"""
    You are an expert in personality psychology. I want you to rate the humility of a person based on their comments. Begin your answer with the score (a number between 0.00 and 1.00).
    Indicators of humility are:
    Sincerity
    Fairness
    Greed-Avoidance
    Modesty

    Here are some examples of accurate scores:
    """


  #execute
  print('Scoring authors based on 5 random comments (seed=42)...')

  for i, author in enumerate(authors):
    random.shuffle(author['comments'])
    working_comments = [comment for comment in author['comments']][:comment_limit]

    few_shot_examples = get_random_examples(df_sorted)
    ex_text1 = few_shot_examples[0][0]
    ex_hum1 = few_shot_examples[0][1]
    ex_text2 = few_shot_examples[1][0]
    ex_hum2 = few_shot_examples[1][1]
    ex_text3 = few_shot_examples[2][0]
    ex_hum3 = few_shot_examples[2][1]
    end_prompt = f'''
    Example 1:
    Comments:{ex_text1}
    Humility:{ex_hum1}
    Example 2:
    Comments:{ex_text2}
    Humility:{ex_hum2}
    Example 3:
    Comments:{ex_text3}
    Humility:{ex_hum3}

    What level of humility does the following person have?
    Comments:{working_comments}
    Humility:
    '''
    final_prompt = prompt + end_prompt
    if i == 0:
      print(f'Example final prompt:\n{final_prompt}')
    answer = generator(final_prompt, max_tokens=4)
    print(f'{i}/{len(authors)}:{answer}')
    author['labels']['humility'] = float(answer)
else:
  print('Skipping model and using local data')
  use_hum_list = True


device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Base prompt:

    You are an expert in personality psychology. I want you to rate the humility of a person based on their comments. Begin your answer with the score (a number between 0.00 and 1.00).
    Indicators of humility are:
    Sincerity
    Fairness
    Greed-Avoidance
    Modesty

    Here are some examples of accurate scores:
    
Scoring authors based on 5 random comments (seed=42)...
Example final prompt:

    You are an expert in personality psychology. I want you to rate the humility of a person based on their comments. Begin your answer with the score (a number between 0.00 and 1.00).
    Indicators of humility are:
    Sincerity
    Fairness
    Greed-Avoidance
    Modesty

    Here are some examples of accurate scores:
    
    Example 1:
    Comments:['Latin from the books of the Laws of England, which taken along with the context, means, that of all whales captured by anybody on the coast of that land, the King, as Honorary Grand Harpooneer, must have the head, and t

##hum list: very long list of values

In [ ]:

if use_hum_list == True:
  hum_string = """
  0.02
  0.05
  0.0
  0.45
  0.0
  0.0
  0.0
  1.0
  0.09
  0.01
  0.1
  0.36
  0.01
  0.0
  0.02
  0.11
  0.0
  0.0
  0.0
  1.0
  0.96
  0.55
  0.01
  0.0
  0.0
  0.12
  0.44
  0.18
  0.01
  0.57
  0.45
  0.99
  0.0
  0.43
  0.1
  0.58
  0.88
  0.01
  0.33
  0.23
  1.0
  1.0
  0.01
  0.21
  0.09
  0.68
  0.01
  0.36
  0.1
  1.0
  0.0
  0.0
  1.0
  0.03
  0.77
  0.05
  0.05
  0.37
  0.0
  0.09
  0.39
  0.09
  0.0
  0.56
  0.24
  0.0
  0.35
  0.0
  1.0
  0.1
  0.07
  0.03
  0.2
  0.19
  0.17
  0.8
  0.19
  0.14
  0.0
  0.06
  0.1
  0.15
  0.0
  0.01
  0.07
  0.0
  0.0
  0.13
  0.03
  0.0
  0.0
  0.16
  0.0
  1.0
  1.0
  0.11
  0.02
  0.61
  0.0
  0.06
  0.99
  0.5
  0.42
  0.07
  0.04
  0.02
  0.19
  0.0
  0.32
  0.51
  0.05
  0.45
  0.28
  0.07
  0.0
  1.0
  0.08
  0.57
  0.0
  0.01
  0.08
  0.34
  0.26
  0.12
  0.03
  0.45
  0.0
  0.0
  0.06
  0.01
  0.28
  0.08
  1.0
  0.0
  1.0
  0.0
  0.0
  0.23
  0.0
  0.07
  0.07
  0.0
  0.0
  0.45
  0.21
  0.24
  0.74
  0.05
  1.0
  0.31
  0.12
  0.04
  0.11
  0.21
  0.01
  0.13
  0.5
  0.0
  0.05
  0.0
  0.56
  1.0
  0.04
  1.0
  0.99
  0.47
  1.0
  0.0
  0.0
  0.85
  1.0
  0.07
  0.0
  0.26
  0.05
  0.0
  0.04
  0.81
  1.0
  0.19
  0.03
  0.84
  0.35
  0.31
  0.51
  0.01
  0.07
  0.45
  0.29
  0.09
  0.12
  0.03
  0.0
  0.01
  0.12
  0.0
  0.08
  0.0
  0.71
  0.46
  0.98
  0.13
  0.18
  0.28
  0.07
  0.33
  0.51
  0.0
  0.15
  0.89
  0.0
  0.21
  0.34
  0.92
  0.84
  0.0
  0.36
  0.0
  0.05
  0.0
  1.0
  0.07
  0.0
  0.0
  0.0
  0.4
  0.11
  0.03
  0.06
  0.97
  0.01
  0.22
  0.0
  0.09
  0.78
  0.15
  0.0
  0.54
  0.05
  0.14
  0.58
  0.0
  0.0
  0.31
  1.0
  0.29
  0.02
  0.99
  0.42
  0.0
  0.05
  0.0
  0.08
  0.0
  0.07
  0.86
  0.0
  0.05
  0.0
  0.78
  0.65
  0.01
  0.01
  0.49
  0.26
  0.01
  0.4
  1.0
  0.05
  0.14
  0.0
  0.75
  0.05
  0.01
  0.05
  0.01
  0.92
  0.04
  0.01
  0.0
  0.07
  0.22
  0.05
  0.03
  0.12
  0.0
  0.0
  0.0
  0.4
  0.39
  0.08
  1.0
  0.01
  0.0
  0.33
  0.47
  0.0
  0.05
  0.0
  0.08
  0.0
  0.16
  0.0
  0.07
  0.0
  0.17
  1.0
  1.0
  0.06
  0.0
  0.19
  0.15
  0.87
  0.06
  0.0
  0.0
  0.05
  0.0
  0.43
  0.27
  0.48
  1.0
  1.0
  0.46
  0.0
  0.01
  0.01
  0.69
  0.0
  0.2
  0.86
  0.01
  0.0
  0.0
  0.8
  1.0
  0.75
  0.0
  0.03
  0.04
  0.13
  0.14
  0.73
  0.0
  0.0
  0.62
  0.32
  0.42
  0.13
  0.0
  0.0
  0.12
  0.08
  1.0
  0.41
  0.0
  0.98
  0.46
  1.0
  1.0
  0.02
  1.0
  0.0
  0.43
  0.9
  0.11
  0.32
  0.32
  0.19
  0.0
  0.0
  0.32
  0.01
  0.0
  0.0
  0.13
  0.05
  0.0
  0.0
  0.06
  0.0
  0.01
  0.67
  0.02
  0.57
  0.78
  0.0
  0.9
  0.0
  0.01
  0.0
  0.05
  0.89
  0.16
  0.96
  0.35
  0.02
  0.0
  0.04
  0.0
  0.04
  0.08
  0.88
  1.0
  1.0
  1.0
  0.51
  0.1
  1.0
  0.0
  0.67
  0.0
  0.16
  0.01
  0.03
  0.1
  0.49
  0.05
  0.11
  0.0
  0.0
  0.01
  0.0
  0.43
  0.0
  0.75
  0.41
  0.06
  0.02
  0.66
  0.86
  0.05
  0.07
  0.45
  0.98
  0.0
  0.14
  0.0
  0.0
  0.4
  0.08
  0.04
  0.78
  0.0
  0.65
  0.11
  0.96
  0.01
  0.64
  0.96
  0.0
  0.0
  0.21
  0.05
  0.28
  1.0
  0.0
  0.24
  0.05
  1.0
  0.0
  0.04
  1.0
  0.01
  0.56
  0.0
  0.0
  0.63
  0.0
  0.07
  0.21
  0.16
  0.0
  0.0
  0.0
  0.15
  0.03
  0.01
  0.0
  0.5
  0.08
  0.0
  0.12
  0.0
  0.05
  0.2
  0.0
  0.14
  0.14
  0.0
  0.4
  0.01
  0.71
  0.0
  0.02
  0.0
  0.0
  0.0
  0.0
  0.01
  0.15
  0.16
  0.06
  0.0
  0.59
  1.0
  0.87
  0.05
  0.0
  0.09
  0.1
  0.18
  0.01
  0.51
  0.09
  0.38
  0.5
  0.05
  0.45
  0.8
  0.07
  0.0
  0.48
  1.0
  0.46
  0.72
  1.0
  0.01
  0.14
  0.0
  0.23
  0.13
  0.11
  0.85
  1.0
  0.99
  0.0
  0.04
  1.0
  0.4
  0.0
  0.0
  0.54
  0.0
  0.0
  0.12
  0.25
  0.0
  0.72
  1.0
  0.0
  0.01
  0.82
  0.01
  0.07
  0.0
  0.0
  0.0
  0.15
  0.0
  0.07
  0.01
  0.65
  0.85
  0.08
  0.09
  0.15
  0.96
  1.0
  0.56
  0.46
  0.95
  0.42
  0.01
  0.32
  0.0
  0.0
  0.41
  0.05
  0.89
  0.08
  0.06
  0.23
  0.35
  0.05
  0.16
  0.0
  0.02
  0.04
  0.0
  0.57
  0.15
  0.05
  0.0
  0.0
  0.07
  0.99
  0.0
  0.07
  0.31
  0.73
  0.92
  0.35
  0.24
  0.45
  0.08
  0.09
  0.05
  1.0
  0.0
  1.0
  0.0
  0.42
  0.0
  0.99
  0.57
  0.23
  0.0
  0.31
  0.88
  0.0
  0.09
  0.96
  0.0
  0.8
  0.03
  0.0
  0.0
  0.07
  0.37
  0.0
  0.05
  0.0
  0.0
  0.0
  0.36
  0.01
  1.0
  0.69
  0.98
  0.15
  0.85
  0.11
  0.0
  0.91
  0.99
  1.0
  0.15
  0.27
  0.0
  0.0
  0.0
  0.0
  0.05
  0.37
  0.0
  0.15
  0.03
  0.0
  0.27
  0.03
  0.02
  0.01
  0.88
  0.29
  0.04
  0.03
  0.04
  0.08
  0.0
  0.02
  0.59
  0.0
  0.04
  0.75
  0.01
  0.09
  0.75
  0.03
  0.05
  0.26
  0.03
  0.45
  0.0
  0.03
  0.1
  0.02
  0.0
  0.71
  0.58
  0.0
  0.0
  0.0
  0.0
  0.0
  0.05
  0.05
  0.51
  0.03
  0.87
  1.0
  0.0
  0.28
  0.0
  0.05
  0.18
  0.28
  0.02
  0.0
  0.25
  0.06
  0.03
  0.13
  0.01
  0.22
  0.08
  0.89
  0.05
  0.01
  0.0
  0.64
  0.36
  0.82
  0.55
  0.99
  0.05
  0.05
  0.05
  0.0
  0.56
  0.0
  0.79
  0.09
  0.0
  0.0
  0.17
  0.04
  0.01
  0.0
  0.69
  0.0
  0.23
  0.1
  0.01
  0.05
  0.07
  0.06
  1.0
  0.13
  0.15
  0.03
  0.79
  1.0
  0.29
  0.07
  0.03
  0.0
  0.57
  0.0
  0.15
  1.0
  0.0
  0.45
  0.1
  0.1
  0.55
  0.06
  0.0
  0.19
  0.0
  0.0
  0.0
  0.56
  0.0
  0.45
  1.0
  0.14
  0.0
  0.8
  0.08
  1.0
  0.15
  0.08
  0.61
  0.63
  0.05
  0.08
  0.07
  0.08
  0.0
  0.0
  0.01
  0.07
  0.79
  0.75
  0.04
  0.0
  0.0
  0.0
  0.01
  0.94
  0.0
  0.76
  0.0
  0.01
  0.05
  0.96
  0.37
  0.14
  0.0
  0.39
  0.87
  0.0
  0.0
  0.79
  0.74
  0.0
  0.49
  0.1
  0.12
  0.0
  0.0
  0.07
  1.0
  0.88
  0.05
  0.0
  0.25
  0.02
  0.83
  1.0
  0.0
  0.15
  1.0
  0.65
  1.0
  0.0
  0.0
  0.01
  0.04
  0.0
  0.0
  0.65
  0.04
  0.05
  0.04
  0.0
  0.68
  0.37
  0.01
  0.0
  0.0
  0.15
  0.0
  0.0
  0.38
  0.99
  0.81
  0.0
  0.0
  0.12
  0.07
  0.86
  0.0
  0.81
  0.12
  0.85
  0.04
  0.0
  0.4
  0.0
  0.01
  0.87
  0.02
  0.08
  0.08
  0.84
  0.0
  0.49
  0.02
  0.0
  0.0
  0.05
  0.0
  0.01
  0.01
  0.08
  0.28
  0.05
  0.08
  0.0
  0.0
  0.01
  0.01
  0.03
  1.0
  0.01
  0.2
  0.33
  0.02
  0.1
  1.0
  0.0
  0.04
  0.05
  0.0
  0.01
  0.0
  0.11
  0.06
  0.6
  0.11
  0.0
  0.0
  0.02
  0.01
  0.0
  0.03
  0.88
  0.9
  0.0
  1.0
  0.05
  1.0
  0.02
  0.0
  0.04
  0.15
  0.89
  0.29
  0.07
  0.0
  0.97
  0.0
  0.12
  0.81
  0.05
  0.0
  0.0
  0.01
  0.0
  0.2
  0.54
  0.8
  0.78
  0.01
  0.29
  0.65
  0.05
  0.98
  0.16
  0.01
  0.03
  0.0
  0.0
  0.72
  0.13
  0.64
  0.0
  1.0
  0.16
  0.18
  0.0
  0.79
  0.51
  1.0
  0.87
  0.02
  0.08
  0.05
  0.0
  0.04
  0.07
  0.53
  0.3
  0.78
  0.34
  0.0
  0.0
  0.04
  0.19
  0.0
  1.0
  0.01
  0.05
  0.0
  0.15
  0.45
  0.01
  1.0
  0.0
  0.03
  0.23
  0.61
  0.45
  0.1
  1.0
  0.01
  1.0
  0.45
  0.02
  0.07
  0.45
  0.06
  0.02
  0.01
  1.0
  0.49
  0.16
  0.15
  0.65
  0.05
  0.23
  0.4
  0.63
  0.67
  0.17
  0.0
  0.0
  0.42
  0.0
  0.01
  0.05
  0.14
  0.9
  0.0
  0.0
  0.0
  0.0
  1.0
  0.11
  0.88
  0.03
  0.0
  0.01
  0.74
  0.0
  0.0
  1.0
  0.26
  0.81
  0.0
  0.0
  0.05
  0.25
  0.0
  0.39
  0.78
  0.05
  0.76
  0.07
  0.0
  0.16
  0.16
  0.0
  0.84
  1.0
  0.0
  0.82
  0.01
  0.0
  0.25
  0.05
  0.0
  0.09
  0.0
  0.35
  0.02
  0.0
  0.01
  0.56
  0.0
  0.01
  0.01
  0.06
  0.46
  0.92
  0.0
  0.07
  0.2
  0.37
  0.23
  0.03
  0.17
  0.88
  0.2
  0.0
  0.07
  0.0
  0.0
  0.05
  0.0
  0.05
  0.15
  0.0
  0.0
  0.05
  0.05
  0.0
  0.0
  0.64
  0.58
  0.05
  0.05
  0.0
  0.1
  0.09
  0.0
  0.01
  0.0
  0.2
  0.05
  0.21
  0.01
  0.0
  0.05
  0.0
  0.27
  0.05
  0.88
  0.0
  0.6
  0.28
  0.06
  0.39
  0.08
  0.01
  0.03
  0.49
  0.0
  0.02
  0.0
  0.07
  0.05
  0.0
  0.73
  0.82
  0.11
  0.18
  0.24
  0.01
  0.0
  1.0
  0.0
  0.0
  0.41
  0.0
  0.12
  0.0
  0.0
  0.18
  0.0
  0.0
  0.17
  0.6
  0.07
  0.0
  0.25
  0.79
  0.0
  0.14
  0.83
  0.91
  0.0
  0.08
  0.0
  0.95
  0.01
  0.0
  0.05
  0.44
  0.0
  0.0
  0.31
  0.14
  0.24
  0.68
  0.22
  0.01
  0.0
  0.05
  0.19
  0.18
  0.16
  0.36
  0.01
  0.0
  0.0
  0.03
  0.0
  0.0
  0.13
  0.56
  0.36
  0.76
  0.0
  0.39
  0.04
  0.55
  0.0
  0.0
  0.01
  0.58
  0.0
  0.0
  0.41
  0.05
  0.09
  0.0
  0.78
  0.09
  0.0
  0.0
  0.09
  0.88
  0.13
  0.0
  0.05
  0.0
  0.0
  0.02
  0.08
  0.0
  0.03
  0.05
  0.01
  0.0
  0.64
  0.0
  1.0
  0.0
  0.57
  0.0
  0.05
  0.14
  0.0
  0.1
  0.01
  0.0
  0.0
  0.0
  0.02
  0.19
  1.0
  0.83
  0.99
  0.02
  0.12
  0.0
  1.0
  0.08
  0.23
  0.25
  0.19
  0.91
  0.09
  0.0
  0.0
  0.03
  0.0
  0.62
  0.43
  0.02
  0.0
  0.0
  0.04
  1.0
  0.15
  0.41
  0.0
  0.0
  0.1
  0.05
  0.02
  0.01
  0.1
  0.24
  0.21
  0.0
  0.6
  0.32
  0.01
  0.01
  0.0
  0.18
  0.28
  0.08
  0.22
  0.53
  0.11
  1.0
  0.18
  0.28
  0.6
  1.0
  0.1
  0.6
  0.24
  0.0
  0.61
  0.17
  0.45
  0.11
  0.13
  0.0
  0.12
  0.0
  0.11
  0.8
  0.49
  0.01
  0.0
  0.08
  0.01
  0.0
  0.01
  0.23
  0.16
  0.05
  0.04
  0.01
  0.08
  0.0
  0.0
  0.05
  0.0
  0.77
  0.05
  0.2
  0.12
  1.0
  0.0
  1.0
  0.0
  1.0
  0.09
  0.0
  1.0
  0.89
  0.77
  0.85
  0.05
  1.0
  0.11
  0.45
  0.67
  0.0
  0.01
  1.0
  0.51
  0.38
  0.04
  0.0
  0.07
  0.18
  0.82
  0.0
  0.0
  0.86
  0.0
  0.0
  0.12
  0.05
  0.43
  0.0
  0.05
  0.75
  0.19
  0.86
  0.59
  0.0
  0.0
  0.12
  0.06
  0.0
  0.0
  0.28
  0.03
  0.43
  0.0
  0.0
  0.99
  1.0
  0.46
  0.0
  0.85
  0.01
  0.0
  0.52
  0.17
  0.25
  0.59
  0.0
  0.07
  0.28
  0.07
  0.83
  0.06
  0.04
  0.31
  1.0
  0.01
  0.04
  0.06
  0.85
  0.08
  0.83
  0.42
  0.41
  0.0
  0.03
  0.86
  0.85
  0.01
  0.12
  0.0
  0.96
  1.0
  0.0
  0.11
  0.82
  0.05
  0.94
  0.04
  0.0
  0.89
  0.39
  0.0
  0.36
  0.07
  0.68
  0.32
  0.07
  0.0
  0.4
  0.0
  0.02
  0.04
  0.0
  0.81
  0.08
  0.01
  0.31
  0.02
  0.02
  0.01
  1.0
  0.01
  0.05
  0.06
  0.0
  0.04
  0.03
  0.05
  0.54
  0.81
  0.87
  0.87
  1.0
  1.0
  0.01
  0.02
  0.57
  0.0
  0.79
  0.56
  1.0
  0.76
  0.35
  0.01
  0.45
  0.01
  0.07
  0.0
  0.0
  0.04
  1.0
  0.55
  0.0
  0.12
  0.0
  0.48
  0.11
  0.67
  0.01
  0.8
  0.43
  0.01
  0.42
  0.95
  0.68
  0.0
  0.06
  0.8
  0.08
  0.16
  """
  hum_list = [float(hum) for hum in hum_string.split()]
  try:
    assert 'authors' in globals(),'pass'
  except:
    try:
      with open(data_path_in, 'r', encoding='utf-8') as f:
        authors = json.load(f)
    except:
      raise Exception('could not load authors or data_path_in')
  for author,hum in zip(authors,hum_list):
    author['labels']['humility'] = hum

  print('Loaded data from list')


Loaded data from list


##end hum list

prepare training data

In [ ]:

def label_to_ordinal(labels): # train is neuro, keeps neuro as neuro
    low_threshold = 0.33
    high_threshold = 0.66
    for key, value in labels.items():
      if value <= low_threshold:
          labels[key] = 0
      elif value <= high_threshold:
          labels[key] = 1
      else:
          labels[key] = 2
    return labels

def change_capital_order(data):
    new_data = []
    for author in data:
        author_id = author['id']
        comments = author['comments']
        labels = author['labels']
        new_labels = {
            'openness': labels['openness'],
            'conscientiousness': labels['conscientiousness'],
            'extraversion': labels['extraversion'],
            'agreeableness': labels['agreeableness'],
            'neuroticism': labels['neuroticism'],
            'humility': labels['humility'] if 'humility' in labels else labels.get('Humility',0)
        }
        new_dict = {
            'id': author_id,
            'comments': comments,
            'labels': new_labels
        }
        new_data.append(new_dict)
    return new_data

# test func
def test_data_transform(path):
    '''
    converts emo stab into neuroticism
    q1-3 into comments as list
    traits -> labels (dict)
    rows into dicts
    '''
    df = pd.read_csv(path)
    cols = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Humility']
    conversion = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    df[cols] = df[cols].apply(lambda col: col.map(conversion))

    #swap es to neuro
    conversion_es_neuro = {
        'low': 2,
        'medium': 1,
        'high': 0
    }
    df['Emotional stability'] = df['Emotional stability'].map(conversion_es_neuro)

    data = []
    for idx, row in df.iterrows():
        comments = [row[col] for col in ['Q1','Q2','Q3']]
        labels = {
            'openness': row['Openness'],
            'conscientiousness': row['Conscientiousness'],
            'extraversion': row['Extraversion'],
            'agreeableness': row['Agreeableness'],
            'neuroticism': row['Emotional stability'],
            'humility': row['Humility']
        }
        new_dict ={
            'id': row['id'],
            'comments': comments,
            'labels': labels
        }
        data.append(new_dict)
    return data

In [ ]:
# training prep
print('Preparing data for training...')
#train data; change floats to ordinal
for author in authors:
    new_labels = label_to_ordinal(author['labels'])
    author['labels'] = new_labels
#train change order to match test
full_train_data = change_capital_order(authors)

#val
try:
  val_path = f'{folder}/data/val_data.csv'
except:
  val_path = r'/content/drive/MyDrive/digital text analysis/NLP/shared task/data/val_data.csv'

test_data = test_data_transform(val_path)
print('\nFun fact! Someone quoted Moby Dick all over the validation set! ^^')
print(f"\'{test_data[24]['comments'][0]}\'")
print(f"\'{test_data[24]['comments'][1]}\'")
print(f"\'{test_data[24]['comments'][2]}\'")

Preparing data for training...

Fun fact! Someone quoted Moby Dick all over my validation set! ^^ (id: 25)
'Latin from the books of the Laws of England, which taken along with the context, means, that of all whales captured by anybody on the coast of that land, the King, as Honorary Grand Harpooneer, must have the head, and the Queen be respectfully presented with the tail. A division which, in the whale, is much like halving an apple; there is no intermediate remainder. Now as this law, under a modified form, is to this day in force in England; and as it offers in various respects a strange anomaly touching the general law of Fast and Loose-Fish, it is here treated of in a separate chapter, on the same courteous principle that prompts the English railways to be at the expense of a separate car, specially reserved for the accommodation of royalty. In the first place, in curious proof of the fact that the above-mentioned law is still in force, I proceed to lay before you a circumstance 

In [ ]:
#save #TEMP #FIX
try:
  if save == True:
    print('\nSaving...')
    with open('train_data.json', 'w', encoding='utf-8') as f:
        json.dump(full_train_data,f)

    with open('val_data.json', 'w', encoding='utf-8') as f:
        json.dump(val_data_holdout,f)
  else:
    pass
except:
  pass

I used this code to check how good the prompting method was... it was ok-ish. (skipped in from_nb)

In [ ]:
if from_nb == 'local':
  %pip install outlines
  #load data
  import pandas as pd
  path_val = r'/content/drive/MyDrive/digital text analysis/NLP/shared task/data/val_data_realvalued.csv'
  df_val = pd.read_csv(path_val)

  def rework_dataframe(df):
    df['Text'] = df[['Q1', 'Q2', 'Q3']].values.tolist()
    df['Humility'] = df['Humility']/100
    df_reworked = df[['Text','Humility']]
    return df_reworked

  df_val = rework_dataframe(df_val)


  from outlines import models

  model_name = "meta-llama/Llama-3.2-3B-Instruct"
  model = outlines.models.transformers(model_name, device='cuda')


  pattern = r'0\.\d{2}|1\.00'
  generator = outlines.generate.regex(model, pattern)


  #few
  for index, row in df_val.iterrows():
    if index == 0:
      ex_text1 = row['Text']
      ex_hum1 = row['Humility']
    elif index == 1:
      ex_text2 = row['Text']
      ex_hum2 = row['Humility']
    elif index == 2:
      ex_text3 = row['Text']
      ex_hum3 = row['Humility']
    else:
      break
  print(ex_text1)
  print(ex_hum1)
  print(ex_text2)
  print(ex_hum2)
  print(ex_text3)
  print(ex_hum3)



  from tqdm import tqdm
  import pandas as pd
  #few shot examples:
  import pandas as pd


  prompt = f"""
  You are an expert in personality psychology. Your task is to rate the humility of a person based on their comments. Begin your answer with the score (a number between 0.00 (not humble) and 1.00 (very humble)).
  Indicators of humility are:
  Sincerity
  Fairness
  Greed-Avoidance
  Modesty

  Here are some examples of accurate scores:
  Example 1:
  Comments:{ex_text1}
  Humility:{ex_hum1}
  Example 2:
  Comments:{ex_text2}
  Humility:{ex_hum2}
  Example 3:
  Comments:{ex_text3}
  Humility:{ex_hum3}

  What level of humility does the following person have?
  """

  pred_labels = []
  for index, row in tqdm(df_val.iterrows()):
    if index <= 2:
      continue
    comments = row['Text']
    final_prompt = prompt + f'Comments:{comments}' + '\n' + 'Humility:'
    answer = generator(final_prompt, max_tokens=4)
    pred_labels.append(answer)
    print(answer)




  true_labels = []
  for index, row in df_val.iterrows():
    if index <= 2:
      continue
    else:
      true_labels.append(row['Humility'])


  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
  import numpy as np

  # Calculate Mean Absolute Error
  mae = mean_absolute_error(true_labels, pred_labels)
  print(f"Mean Absolute Error: {mae}")

  # Calculate Mean Squared Error
  mse = mean_squared_error(true_labels, pred_labels)
  print(f"Mean Squared Error: {mse}")

  # Calculate Root Mean Squared Error
  rmse = np.sqrt(mse)
  print(f"Root Mean Squared Error: {rmse}")

  # Calculate R-squared
  r2 = r2_score(true_labels, pred_labels)
  print(f"R-squared: {r2}")

else:
  print('Skipped testing portion of notebook.')